## Imports

In [1]:
#Delete me
# Settings for notebook visualization
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
%matplotlib inline
from IPython.core.display import HTML
HTML("""<style>.output_png img {display: block;margin-left: auto;margin-right: auto;text-align: center;vertical-align: middle;} </style>""")

In [2]:
# Necessary imports
import os
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import DivergingNorm
import seaborn as sns
#import quantstats as qs
#import statistics as st

In [3]:
# # Other settings

# # Settings for plot visualization
# plt.style.use('seaborn-darkgrid')

# #plt.rcParams.keys()
# plt.rcParams['figure.dpi'] = 200
# plt.rcParams["figure.figsize"] = (12,3.5) #(12,5)
# plt.rcParams['axes.grid'] = True
# plt.rcParams['grid.linewidth'] = 0.4
# #plt.rcParams['xtick.label.allignment'] = 'center'

# plt.rcParams['xtick.bottom'] = plt.rcParams['ytick.labelright'] = True
# plt.rcParams['ytick.left'] = plt.rcParams['ytick.right'] = True

# plt.rcParams['lines.linewidth'] = 1.2
# #plt.rcParams['lines.markersize'] = 0.5
# plt.rcParams['patch.edgecolor'] = 'k' # Legend border 
# plt.rcParams['legend.facecolor'] = 'w'
# plt.rcParams["legend.frameon"] = True

# np.set_printoptions(edgeitems=40, linewidth=1000)

# pd.set_option("display.precision", 6)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
# #print("Notebook parameters set correctly")

In [4]:
os.chdir("/Users/Sergio/Documents/Master_QF/Thesis/Code/Algorithmic Strategies/")

In [5]:
# ini_equity_default = 100
# commision_default = 0.000111538462 # 2/130000 + 12.5/130000
# #commision_default = 0.0005 # Slightly Bbgger commision, for better visualization
# # 0.01 = 1% of the cummulative return (equity)

## Plots

In [6]:
def show_plot(df, benchmark=True, position=True, figsize=(12,3.6), with_legend=False, with_signals=False): #(12,5)
    df_plot = df.copy()
    first_day = df_plot.index[0]
    last_day = df_plot.index[-1]
    
    fmt = '%Y-%m-%d' if first_day.year == last_day.year else '%Y'
        
    columns_colors_ax1 = [('Strat_cum_ret', 'b')]
    
    title = 'MA crossover strategy ({} : {})'.format(first_day.strftime(fmt), last_day.strftime(fmt))

    if (benchmark == True):
        title = 'SP500 vs ' + title
        columns_colors_ax1.append(('Market_cum_ret', 'k'))

    # PLOT
    if position == True:
        import matplotlib.dates as mdates
        
        fig, ax1 = plt.subplots(figsize=figsize)
        ax2 = ax1.twinx()

        [ax1.plot(df_plot.index, df_plot[column], label=column, color=color) for (column, color) in columns_colors_ax1[::-1]]
        
        columns_label_colors_ax2 = [('Strat_position', 'Strategy position', 'r')]
        columns_label_colors_ax2 = [('Long_only', 'Only long position allowed', 'tab:brown'), ('Strat_position', 'Strategy position', 'g')]
        df_plot['Long_only'] = df_plot['Long_only'] - 0.03
        
        [ax2.scatter(df_plot.index, df_plot[column], marker='s', s=1, label=lab, color=color) for (column, lab, color) in columns_label_colors_ax2]            
        ax2.set(ylabel='Strategy Position', yticks=np.arange(-1.0,1.01,1))
        ax2.set_ylim([-1.1,1.1])
        ax2.tick_params(axis='y', direction='out', length=5)
        
        ax2.grid(False)
        
        handles, labels = ax2.get_legend_handles_labels()
        if with_legend == True:
            ax2.legend(handles[::-1], labels[::-1], loc='upper left', bbox_to_anchor=(0.55, -0.08), borderpad=0.5, markerscale=3) #borderpad=1
        
    else:
        [ax1.plot(df_plot.index, df_plot[column], label=column, color=color) for (column, color) in columns_colors_ax1[::-1]]

    from matplotlib import ticker
    
    # X-AXES
    # Write a max of 20 major locators in x axis
    n_years = last_day.year - first_day.year
    freq = 1 if (n_years <= 20) else 5
    years = mdates.YearLocator(freq)       
    yearFmt = mdates.DateFormatter('%Y')
    # Add the locators to the axis
    ax1.xaxis.set_major_locator(years)
    ax1.xaxis.set_major_formatter(yearFmt) # Add tick with every freq years
    ax1.xaxis.set_minor_locator(mdates.YearLocator(1)) # every year
    ax1.tick_params(axis='x', direction='out', which='major', length=4)
    ax1.tick_params(axis='x', direction='out', which='minor', length=2)  
    
    loc = full_df.index.get_loc(first_day)
    first_tick = full_df.index.values[loc-1]
    loc = full_df.index.get_loc(last_day)
    loc = loc if (loc == full_df.index.size - 1) else loc + 1 # Necessary to add one day checking for index size
    last_tick = full_df.index.values[loc]
    ax1.set_xlim([first_tick, last_tick])
    
    # Y-AXES
    ax1.set(ylabel='Value (USD)', title=title)
    ax1.tick_params(axis='y', direction='out', length=4)
    #ax1.yaxis.set_minor_locator(ticker.MultipleLocator(250))
    #ax1.tick_params(axis='y', direction='out', which='minor', length=2)
    
    # LEGEND
    if with_legend == True:
        # Put a legend below current axis, reversing the printing order and incresing linewidth
        handles, labels = ax1.get_legend_handles_labels()
        repl = {'Strat_cum_ret':'Strategy', 'Market_cum_ret':'Benchmark', 'Strat_position':'Strategy position'}
        repl = {'Strat_cum_ret':'Strategy', 'Market_cum_ret':'Benchmark', 'Long_only':'Only long position allowed', 'Strat_position':'Strategy position'}
        labels = [repl.get(n, n) for n in labels]
        leg1 = ax1.legend(handles[::-1], labels[::-1], loc='upper right', bbox_to_anchor=(0.45, -0.08), borderpad=0.5)
        [legobj.set_linewidth(3) for legobj in leg1.legendHandles]
    
    if with_signals == True:
        buy_signal = df_plot['Strat_position'].shift(periods=-1) > df_plot['Strat_position']
        buy_marker = df_plot['Market_cum_ret']
        buy_marker = buy_marker[buy_signal]
        buy_dates = df_plot.index[buy_signal]
        sell_signal = df_plot['Strat_position'].shift(periods=-1) < df_plot['Strat_position']
        sell_marker = df_plot['Market_cum_ret']
        sell_marker = sell_marker[sell_signal]
        sell_dates = df_plot.index[sell_signal]
        ax1.scatter(buy_dates, buy_marker, marker='^', color='green', label='Buy');
        ax1.scatter(sell_dates, sell_marker, marker='>', color='red', label='Exit');

    # 
    ax1.set_zorder(1)
    ax1.patch.set_visible(False)
    ax2.set_zorder(0)
    ax2.patch.set_visible(True)

In [7]:
def plot_sp500_with_ma_signals(df, fast_ma, slow_ma):
    first_day = df.index[0]
    last_day = df.index[-1]
    
    fmt = '%Y-%m-%d' if first_day.year == last_day.year else '%Y'

    df['fast_ma'] = full_df['Close'].rolling(window=fast_ma).mean()[first_day:last_day]
    df['slow_ma'] = full_df['Close'].rolling(window=slow_ma).mean()[first_day:last_day]
    df['diff'] = df['fast_ma'].sub(df['slow_ma'])
    df['long_signal'] = (df['diff'] > 0) & (df['diff'].shift(1).fillna(-1) <= 0)
    df['exit_signal'] = (df['diff'] <= 0) & (df['diff'].shift(1) > 0)

    buy_marker = df['fast_ma'] * df['long_signal'] - (df['fast_ma'].max()*.01)
    buy_marker = buy_marker[df['long_signal']]
    buy_dates = df.index[df['long_signal']]
    exit_marker = df['fast_ma'] * df['exit_signal'] - (df['fast_ma'].max()*.01)
    exit_marker = exit_marker[df['exit_signal']]
    exit_dates = df.index[df['exit_signal']]

    title = "SP500 with MAs of {}-{} ({} : {})".format(fast_ma, slow_ma, first_day.strftime(fmt), last_day.strftime(fmt))
    
    fig = plt.figure();
    ax = df[['Close', 'fast_ma', 'slow_ma']].plot(title=title,
                                             figsize=(14,6), 
                                             color=('k', 'r', 'y'));
    
    ax.scatter(buy_dates, buy_marker, marker='^', color='green', label='Buy');
    ax.scatter(exit_dates, exit_marker, marker='>', color='blue', label='Exit');

    ax.legend(["SP500", 'MA '+ str(fast_ma), 'MA '+ str(slow_ma)]);
    ax.tick_params(axis='x', direction='out', length=3, labelrotation=0);
    ax.tick_params(axis='y', direction='out', length=3);
    plt.xticks(horizontalalignment="center");

In [8]:
def show_oos_plot(results_df, with_legend=False, with_signals=False):
    show_plot(results_df, with_legend=with_legend, with_signals=with_signals)

In [9]:
# Plots a heatmap with data from a matrix. 
def show_heatmap(data, market_ir, plot_title, x_title, x_values, y_title, y_values):
    # Flip matrix vertically for better visualization
    data = np.flip(data, axis=0)
    #data = np.nan_to_num(data, nan=market_ir)
    fig, ax = plt.subplots(figsize=(3, 3)) #11,9

    rdgn = sns.diverging_palette(h_neg=10, h_pos=130, as_cmap=True, s=80, l=50)
#    divnorm = DivergingNorm(vmin=data.min(), vcenter=0, vmax=data.max())
#    sns.heatmap(data, cmap=rdgn, norm=divnorm, annot=True, fmt ='.2', 
    fig = sns.heatmap(data, cmap=rdgn, annot=True, fmt =".3f", annot_kws={"fontsize":5}, 
                vmin=-2.0, center=0, vmax=2.0,
                linecolor='black', cbar=True, ax=ax,
                xticklabels=x_values, yticklabels=np.flip(y_values))
    cbar = fig.collections[0].colorbar
    cbar.ax.tick_params(labelsize=5);
    ax.yaxis.tick_left();
    plt.xlabel(x_title, fontsize=6);
    plt.ylabel(y_title, fontsize=6);
    plt.title(plot_title, fontsize=7);
    plt.xticks(fontsize=6, rotation=0);
    plt.yticks(fontsize=6, rotation=0);

In [22]:
# Plots a heatmap with data from a matrix. 
def show_both_heatmaps(data_robust, data_hist, market_ir, plot_title, left_plot_title, right_plot_title, 
                           x_title, x_values, y_title, y_values):
    # Flip matrix vertically for better visualization
    data_robust = np.flip(data_robust, axis=0)
    data_hist = np.flip(data_hist, axis=0)

#     data_robust = np.multiply(np.flip(data_robust, axis=0), 100)
#     data_hist = np.multiply(np.flip(data_hist, axis=0), 100)

    #data = np.nan_to_num(data, nan=market_ir)
    fig, ax = plt.subplots(ncols=2, figsize=(11, 4.5)) #11,9
    fig.subplots_adjust(wspace=0.2)
    fig.suptitle("Information Ratio In Sample (" + plot_title +")", fontsize=9)
    ax[0].set_title(left_plot_title, fontsize=7)
    ax[1].set_title(right_plot_title, fontsize=7)
    
    rdgn = sns.diverging_palette(h_neg=10, h_pos=130, as_cmap=True, s=80, l=50)
    
    smallest = 0 if data_robust.min() > 0 else data_robust.min()
    #middle = market_ir if market_ir != np.nan else 0 #market_ir
    middle = data_robust[0,0] #market_ir

    largest = data_robust.max()
    sns.heatmap(data_robust, cmap=rdgn, annot=True, fmt =".2f", annot_kws={"fontsize":5}, 
                vmin=smallest, center=middle, vmax=largest,
                linecolor='black', cbar=False, ax=ax[0],
                xticklabels=x_values, yticklabels=np.flip(y_values))
    fig.colorbar(ax[0].collections[0], ax=ax[0], location="left", use_gridspec=False, pad=0.1)

    smallest = 0 if data_hist.min() > 0 else data_hist.min()
    middle = data_hist[0,0] #market_ir
    largest = data_hist.max()
    sns.heatmap(data_hist, cmap=rdgn, annot=True, fmt =".2f", annot_kws={"fontsize":5}, 
                vmin=smallest, center= middle, vmax=largest,
                linecolor='black', cbar=False, ax=ax[1],
                xticklabels=x_values, yticklabels=np.flip(y_values))
    fig.colorbar(ax[1].collections[0], ax=ax[1], location="right", use_gridspec=False, pad=0.1)
    ax[1].yaxis.set_label_position("right")
    #ax[1].yaxis.tick_right()
    
    for a in ax:
        a.collections[0].colorbar.ax.tick_params(labelsize=5)
        a.set_xlabel(x_title, fontsize=7)
        a.set_ylabel(y_title, fontsize=7);
        a.tick_params(rotation=0, labelsize=6)
    

In [12]:
def add_data(data, new_data, name):
    columns = pd.MultiIndex.from_product([[name], metrics])
    new_df = pd.DataFrame(new_data, index=index, columns=columns)
    concatenate = pd.concat([data, new_df], axis=1)

    return concatenate

In [13]:
def save_report(returns, benchmark="SPY"):
#     color = 'k'
#     plt.rcParams['text.color'] = color
#     plt.rcParams['legend.facecolor'] = 'w'
#     plt.rcParams['xtick.color'] = color
#     plt.rcParams['ytick.color'] = color
#     plt.rcParams['axes.labelcolor'] = color
    
    #     plt.rcParams['text.color'] = 'w'
    #     plt.rcParams['legend.facecolor'] = '#2f3540'
    #     plt.rcParams['xtick.color'] = 'w'
    #     plt.rcParams['ytick.color'] = 'w'
    #     plt.rcParams['axes.labelcolor'] = 'w'
    qs.reports.html(returns, benchmark)